# Pertemuan 13
- OpenCV Video Decoding
- OpenCV Image Rendering
- Realtime Object Segmentation - Contour Based
- Realtime Object Segmentation - OpenCV Tracking API
___
### Maximizing Jetson Nano Perfomance

In [ ]:
# sudo nvpmodel -m 0
# sudo jetson_clocks

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

-  load some utility function

In [2]:
from gst_file import gst_file_loader
from draw_utils import draw_ped

In [ ]:
# check OpenCV Version

cv2.__version__

___
# 1. OpenCV Video Decoding - GStreamer vs FFMPEG

In [ ]:
# load video file using GStreamer
#cap = cv2.VideoCapture(gst_file_loader("video_a.mp4"), cv2.CAP_GSTREAMER)  # backend GSTREAMER

gst = "rtspsrc location=https://192.168.0.103:8081/index.jpg latency=0 ! rtph264depay ! h264parse ! omxh264dec ! videoconvert ! appsink"
cap = cv2.VideoCapture(gst, cv2.CAP_GSTREAMER)
# cap = cv2.VideoCapture("video_a.mp4")  # backend FFMPEG
# cap.set(cv2.CAP_PROP_ORIENTATION_AUTO, 1) # applicable for backend FFMPEG only

times = []
while cap.isOpened() : 
    e1 = cv2.getTickCount()
    ret, frame = cap.read()
    if not ret : 
        break 

    cv2.imshow("window", frame)
    if cv2.waitKey(1) == ord("q"):
        break
    e2 = cv2.getTickCount()
    times.append((e2 - e1)/ cv2.getTickFrequency())

time_avg = np.array(times).mean()
print("Average execution time : %.4fs" % time_avg)
print("Average FPS : %.2f" % (1/time_avg))

cv2.destroyAllWindows()

## Result Benchmark Video Decoding
- Source video : `video_a.mp4` 
- `cv2.VideoCapture()` with backend FFMPEG
    - Average execution time : 0.1627s
    - Average FPS : 6.15
- `cv2.VideoCapture()` with backend GStreamer
    - Average execution time : 0.1587s
    - Average FPS : 6.30
- `cv2.VideoCapture()` with backend FFMPEG + Jetson maximized performance
    - Average execution time : 0.1615s
    - Average FPS : 6.19
- `cv2.VideoCapture()` with backend GStreamer + Jetson maximized performance
    - Average execution time : 0.1347s
    - Average FPS : 7.42

___
# 2. OpenCV Image Rendering - Standard vs OPENGL
- To use OpenGL in Jetson Nano try to build OpenCV with OpenGL Enable by following [this tutorial](https://yunusmuhammad007.medium.com/build-and-install-opencv-4-5-3-on-jetson-nano-with-cuda-opencl-opengl-and-gstreamer-enable-6dc7141be272). <br><br>
- Install `mesa-utils` to help us check OpenGL available on our machine (Jetson Nano)
    ```
    sudo apt-get install mesa-utils
    ```

In [ ]:
#  ---- Check OpenGL Version ----
# glxinfo | grep OpenGL
#
# 
# ---- Check DISPLAY variable ----
# echo $DISPLAY
#
#
# ---- Enable NVIDIA OpenGL ----
# sudo service gdm3 stop
# sudo X
# export DISPLAY=:0
# xrandr

In [ ]:
%env DISPLAY=:0

In [ ]:
# EXAMPLE Play Video Stream + OpenGL Image Rendering

window_name = "Window"
cv2.namedWindow(window_name, flags=cv2.WINDOW_OPENGL)    # Window with OpenGL

# load video file using GStreaqmer
cap = cv2.VideoCapture(gst_file_loader("video_a.mp4"), cv2.CAP_GSTREAMER)  # backend GSTREAMER

# cap = cv2.VideoCapture("video_a.mp4", cv2.CAP_FFMPEG)  # backend FFMPEG
# cap.set(cv2.CAP_PROP_ORIENTATION_AUTO, 1) # applicable for backend FFMPEG only

times = []
while cap.isOpened() : 
    e1 = cv2.getTickCount()
    ret, frame = cap.read()
    if not ret : 
        break 

    cv2.imshow(window_name, frame)
    if cv2.waitKey(1) == ord("q"):
        break
    e2 = cv2.getTickCount()
    times.append((e2 - e1)/ cv2.getTickFrequency())

time_avg = np.array(times).mean()
print("Average execution time : %.4fs" % time_avg)
print("Average FPS : %.2f" % (1/time_avg))

cv2.destroyAllWindows()

## Result Benchmark 
- `cv2.VideoCapture()` with backend GStreamer + Jetson maximized performance + OpenGL Image Rendering
    - Average execution time : 0.0332s
    - Average FPS : 30.09


____
# 3. Realtime Object Segmentation - Contour Based
- This is based on example in [section 6](https://github.com/Muhammad-Yunus/Jetson-Nano-OpenCV-Learn/tree/main/pertemuan_6).

## 3.1 Example 6 From Section 6

In [ ]:
######### EXAMPLE 6 FROM SECTION 6 : (part_b.jpg) ##########
#
# Detect Contour from Binary Image (Range Thresholding) + 
# Background removal + 
# Contour Filter by hierarchy + 
# Contour Filter by Contour Property
# Count Child Contour for each parent
# Draw_Ped on object
#
############################################################


img = cv2.imread("part_b.jpg")

#convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# define range of gray color in HSV 
lower_gray = np.array([0, 0, 20])       # (part_a.jpg : [0, 0, 20] | part_b.jpg : [0, 0, 10])
upper_gray = np.array([180, 100, 150])  # (part_a.jpg : [180, 100, 150]| part_b.jpg : [180, 100, 170])

# Threshold the HSV image to get only gray colors
mask = cv2.inRange(hsv.copy(), lower_gray, upper_gray)

res = cv2.bitwise_and(img, img, mask=mask)

contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

object_contour = {}
object_count = {}
object_id = 0

THRESHOLD_COUNT = 22 # min number of child contour  (part_a.jpg : 22 | part_b.jpg : 2)
MIN_AREA = 200 # minimum number of pixel to be counted to reject small contour

# Contour Property parameter for parent contour
MAX_ASPECT_RATIO = 0.3 # (part_a.jpg : 0.3 | part_b.jpg : 0.5)
MIN_ASPECT_RATIO = 0.1 # (part_a.jpg : 0.1 | part_b.jpg : 0.3)
MIN_EXTENT = 0.4

# Contour Property parameter for child contour
MAX_ASPECT_RATIO_CHILD = 1.5
MIN_ASPECT_RATIO_CHILD = 0.5
MIN_EXTENT_CHILD = 0.4

for cnt, hrcy in zip(contours, hierarchy[0]):
    # find contour Area & boungin Rect
    area = cv2.contourArea(cnt)
    x, y, w, h = cv2.boundingRect(cnt)

    # calculate aspectRatio & extent
    aspectRatio = float(w)/h 
    rect_area = w*h
    extent = float(area)/rect_area
    
    # filter a small contour
    if area <= MIN_AREA:
        continue 
    
   # Find All Extreme Outer Contour [BLUE]
    if hrcy[3] == -1 :   
        if aspectRatio < MAX_ASPECT_RATIO and aspectRatio > MIN_ASPECT_RATIO and extent > MIN_EXTENT:      
            cv2.drawContours(res, cnt, -1, (255,0,0), 3)
            
            object_contour["object_%d" % object_id] = cnt # insert parent contour
            object_count["object_%d" % object_id] = 0 # set initinal count 0
            object_id += 1

    # Find All child contour [GREEN]
    if hrcy[3] != -1 :  
        if aspectRatio < MAX_ASPECT_RATIO_CHILD and aspectRatio > MIN_ASPECT_RATIO_CHILD and extent > MIN_EXTENT_CHILD:    
            cv2.drawContours(res, cnt, -1, (0,255,0), 3)

            for obj_name in object_contour:
                # find the child contour on wich parrent contour
                if cv2.pointPolygonTest(object_contour[obj_name], (x, y), measureDist=True) > 0 :
                    object_count[obj_name] += 1


for obj_name in object_count:
    x, y, w, h = cv2.boundingRect(object_contour[obj_name])
    # check if number of child contour inside parrent less than threshold count 
    if object_count[obj_name] < THRESHOLD_COUNT :
        img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                    font_size=0.7, alpha=0.6, bg_color=(0,0,255), ouline_color=(0,0,255), text_color=(0,0,0))
    else :
        img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                    font_size=0.7, alpha=0.6, bg_color=(0,255,0), ouline_color=(0,255,0), text_color=(0,0,0))        

plt.figure(figsize=(20,30))
plt.subplot(1,2,1)
plt.imshow(img[:,:,::-1])
plt.title("Original Image")

plt.subplot(1,2,2)
plt.imshow(res[:,:,::-1])
plt.title("Result")

plt.show()

## 3.2 Implementation in Video Stream

In [ ]:
# EXAMPLE 3.2.1 | Apply Contour Based Visual Inspection Engine for Video Stream

# Uncomment this if using OpenGL for image rendering
window_name_1 = "Detected Object"
window_name_2 = "Detected Contour"
cv2.namedWindow(window_name_1, flags=cv2.WINDOW_OPENGL)    # Window with OpenGL
cv2.namedWindow(window_name_2, flags=cv2.WINDOW_OPENGL)    # Window with OpenGL

# define range of gray color in HSV 
lower_gray = np.array([0, 0, 20])       # (part_a.jpg : [0, 0, 20] | part_b.jpg : [0, 0, 10])
upper_gray = np.array([180, 100, 150])  # (part_a.jpg : [180, 100, 150]| part_b.jpg : [180, 100, 170])

THRESHOLD_COUNT = 22 # min number of child contour  (part_a.jpg : 22 | part_b.jpg : 2)
MIN_AREA = 100 # minimum number of pixel to be counted to reject small contour

# Contour Property parameter for parent contour
MAX_ASPECT_RATIO = 0.3 # (part_a.jpg : 0.3 | part_b.jpg : 0.5)
MIN_ASPECT_RATIO = 0.1 # (part_a.jpg : 0.1 | part_b.jpg : 0.3)
MIN_EXTENT = 0.4

# Contour Property parameter for child contour
MAX_ASPECT_RATIO_CHILD = 1.5
MIN_ASPECT_RATIO_CHILD = 0.5
MIN_EXTENT_CHILD = 0.4


# load video file using GStreamer
cap = cv2.VideoCapture(gst_file_loader("video_a.mp4", useRotate90=True), cv2.CAP_GSTREAMER)  # backend GSTREAMER 

# cap = cv2.VideoCapture("video_a.mp4", cv2.CAP_FFMPEG)  # backend FFMPEG
# cap.set(cv2.CAP_PROP_ORIENTATION_AUTO, 1) # applicable for backend FFMPEG only

times = []
while cap.isOpened() :
    object_contour = {}
    object_count = {}
    object_id = 0 

    e1 = cv2.getTickCount()
    ret, img = cap.read()
    if not ret : 
        break 

    #convert to hsv
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Threshold the HSV image to get only gray colors
    mask = cv2.inRange(hsv.copy(), lower_gray, upper_gray)

    # apply eroding into mask
    kernel = np.ones((2,2),np.uint8)
    erosion = cv2.erode(mask, kernel, iterations = 2)

    # apply bitwise operation (for background removal), if needed.
    res = cv2.bitwise_and(img, img, mask=erosion)

    # find contour from range thresholding
    contours, hierarchy = cv2.findContours(erosion, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    for cnt, hrcy in zip(contours, hierarchy[0]):
        # find contour Area & boungin Rect
        area = cv2.contourArea(cnt)
        x, y, w, h = cv2.boundingRect(cnt)

        # calculate aspectRatio & extent
        aspectRatio = float(w)/h 
        rect_area = w*h
        extent = float(area)/rect_area
        
        # filter a small contour
        if area <= MIN_AREA:
            continue 
        
        # Find All Extreme Outer Contour [BLUE]
        if hrcy[3] == -1 :   
            if aspectRatio < MAX_ASPECT_RATIO and aspectRatio > MIN_ASPECT_RATIO and extent > MIN_EXTENT:      
                cv2.drawContours(res, cnt, -1, (255,0,0), 2)
                
                object_contour["object_%d" % object_id] = cnt # insert parent contour
                object_count["object_%d" % object_id] = 0 # set initinal count 0
                object_id += 1

        # Find All child contour [GREEN]
        if hrcy[3] != -1 :  
            if aspectRatio < MAX_ASPECT_RATIO_CHILD and aspectRatio > MIN_ASPECT_RATIO_CHILD and extent > MIN_EXTENT_CHILD:    
                cv2.drawContours(res, cnt, -1, (0,255,0), 2)

                for obj_name in object_contour:
                    # find the child contour on wich parrent contour
                    if cv2.pointPolygonTest(object_contour[obj_name], (x, y), measureDist=True) > 0 :
                        object_count[obj_name] += 1


    for obj_name in object_count:
        x, y, w, h = cv2.boundingRect(object_contour[obj_name])
        # check if number of child contour inside parrent less than threshold count 
        if object_count[obj_name] < THRESHOLD_COUNT :
            img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                        font_size=0.4, alpha=0.6, bg_color=(0,0,255), ouline_color=(0,0,255), text_color=(0,0,0))
        else :
            img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                        font_size=0.4, alpha=0.6, bg_color=(0,255,0), ouline_color=(0,255,0), text_color=(0,0,0)) 

    cv2.imshow(window_name_1, img)
    cv2.imshow(window_name_2, res)
    if cv2.waitKey(1) == ord("q"):
        break
    e2 = cv2.getTickCount()
    times.append((e2 - e1)/ cv2.getTickFrequency())

time_avg = np.array(times).mean()
print("Average execution time : %.4fs" % time_avg)
print("Average FPS : %.2f" % (1/time_avg))

cv2.destroyAllWindows()

## Result Benchmark
- Contour Based Visual Inspection Engine for Video Stream (FFMPEG Backend) + Jetson Maximized Performance
    - Average execution time : 0.2012s
    - Average FPS : 4.97
- Contour Based Visual Inspection Engine for Video Stream (GStreamer Backend) + Jetson Maximized Performance
    - Average execution time : 0.1938s
    - Average FPS : 5.16
- Contour Based Visual Inspection Engine for Video Stream (FFMPEG Backend) + Jetson Maximized Performance + OpenGL Image Rendering
    - Average execution time : 0.0604s
    - Average FPS : 16.55

## 3.3 CUDA Implemetation

In [ ]:
########## EXAMPLE CUDA Implementation from Section 6 ##########

# load image in Host memory
img = cv2.imread("part_b.jpg")
h, w, c = img.shape

# GPU memory initialization
img_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
img_GpuMat.create((w, h), cv2.CV_8UC3) # cv2.CV_8UC3 -> 8bit image 3 channel
hsv_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
hsv_GpuMat.create((w, h), cv2.CV_8UC3) # cv2.CV_8UC3 -> 8bit image 3 channel
h_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
h_GpuMat.create((w, h), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
s_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
s_GpuMat.create((w, h), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
v_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
v_GpuMat.create((w, h), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
mask_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
mask_GpuMat.create((w, h), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
res_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
res_GpuMat.create((w, h), cv2.CV_8UC3) # cv2.CV_8UC3 -> 8bit image 3 channel

# upload to GPU memory
img_GpuMat.upload(img)

# CUDA convert to hsv
cv2.cuda.cvtColor(img_GpuMat, cv2.COLOR_BGR2HSV, hsv_GpuMat)

# split HSV GPU Mat
cv2.cuda.split(hsv_GpuMat, [h_GpuMat, s_GpuMat, v_GpuMat])

# CUDA Threshold the V(20,150) ~ HSV GPU Mat to get only gray colors
cv2.cuda.inRange(v_GpuMat, 20, 150, mask_GpuMat)

# CUDA bitwise operation
cv2.cuda.bitwise_not(img_GpuMat, res_GpuMat, mask=mask_GpuMat) # apply bitwise NOT to original image -> result image
cv2.cuda.bitwise_not(res_GpuMat, res_GpuMat, mask=mask_GpuMat) # apply bitwise NOT to result image 

# Download Matrix to Host Memory                                                               
mask = mask_GpuMat.download()
res = res_GpuMat.download()

# find contour
contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

object_contour = {}
object_count = {}
object_id = 0

THRESHOLD_COUNT = 22 # min number of child contour  (part_a.jpg : 22 | part_b.jpg : 2)
MIN_AREA = 200 # minimum number of pixel to be counted to reject small contour

# Contour Property parameter for parent contour
MAX_ASPECT_RATIO = 0.3 # (part_a.jpg : 0.3 | part_b.jpg : 0.5)
MIN_ASPECT_RATIO = 0.1 # (part_a.jpg : 0.1 | part_b.jpg : 0.3)
MIN_EXTENT = 0.4

# Contour Property parameter for child contour
MAX_ASPECT_RATIO_CHILD = 1.5
MIN_ASPECT_RATIO_CHILD = 0.5
MIN_EXTENT_CHILD = 0.4

for cnt, hrcy in zip(contours, hierarchy[0]):
    # find contour Area & boungin Rect
    area = cv2.contourArea(cnt)
    x, y, w, h = cv2.boundingRect(cnt)

    # calculate aspectRatio & extent
    aspectRatio = float(w)/h 
    rect_area = w*h
    extent = float(area)/rect_area
    
    # filter a small contour
    if area <= MIN_AREA:
        continue 
    
   # Find All Extreme Outer Contour [BLUE]
    if hrcy[3] == -1 :   
        if aspectRatio < MAX_ASPECT_RATIO and aspectRatio > MIN_ASPECT_RATIO and extent > MIN_EXTENT:      
            cv2.drawContours(res, cnt, -1, (255,0,0), 3)
            
            object_contour["object_%d" % object_id] = cnt # insert parent contour
            object_count["object_%d" % object_id] = 0 # set initinal count 0
            object_id += 1

    # Find All child contour [GREEN]
    if hrcy[3] != -1 :  
        if aspectRatio < MAX_ASPECT_RATIO_CHILD and aspectRatio > MIN_ASPECT_RATIO_CHILD and extent > MIN_EXTENT_CHILD:    
            cv2.drawContours(res, cnt, -1, (0,255,0), 3)

            for obj_name in object_contour:
                # find the child contour on wich parrent contour
                if cv2.pointPolygonTest(object_contour[obj_name], (x, y), measureDist=True) > 0 :
                    object_count[obj_name] += 1


for obj_name in object_count:
    x, y, w, h = cv2.boundingRect(object_contour[obj_name])
    # check if number of child contour inside parrent less than threshold count 
    if object_count[obj_name] < THRESHOLD_COUNT :
        img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                    font_size=0.7, alpha=0.6, bg_color=(0,0,255), ouline_color=(0,0,255), text_color=(0,0,0))
    else :
        img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                    font_size=0.7, alpha=0.6, bg_color=(0,255,0), ouline_color=(0,255,0), text_color=(0,0,0)) 

plt.figure(figsize=(20,30))
plt.subplot(1,2,1)
plt.imshow(img[:,:,::-1])
plt.title("Original Image")

plt.subplot(1,2,2)
plt.imshow(res[:,:,::-1])
plt.title("Result")

In [ ]:
# EXAMPLE 3.3.1 | Apply CUDA Contour Based Visual Inspection Engine for Video Stream

# Uncomment this if using OpenGL for image rendering
window_name_1 = "Detected Object"
window_name_2 = "Detected Contour"
cv2.namedWindow(window_name_1, flags=cv2.WINDOW_OPENGL)    # Window with OpenGL
cv2.namedWindow(window_name_2, flags=cv2.WINDOW_OPENGL)    # Window with OpenGL

THRESHOLD_COUNT = 22 # min number of child contour  (part_a.jpg : 22 | part_b.jpg : 2)
MIN_AREA = 100 # minimum number of pixel to be counted to reject small contour

# Contour Property parameter for parent contour
MAX_ASPECT_RATIO = 0.3 # (part_a.jpg : 0.3 | part_b.jpg : 0.5)
MIN_ASPECT_RATIO = 0.1 # (part_a.jpg : 0.1 | part_b.jpg : 0.3)
MIN_EXTENT = 0.4

# Contour Property parameter for child contour
MAX_ASPECT_RATIO_CHILD = 1.5
MIN_ASPECT_RATIO_CHILD = 0.5
MIN_EXTENT_CHILD = 0.4


#load video file using GStreamer
cap = cv2.VideoCapture(gst_file_loader("video_a.mp4", useRotate90=True), cv2.CAP_GSTREAMER)  # backend GSTREAMER 

# cap = cv2.VideoCapture("video_a.mp4", cv2.CAP_FFMPEG)  # backend FFMPEG
# cap.set(cv2.CAP_PROP_ORIENTATION_AUTO, 1) # applicable for backend FFMPEG only


# GPU memory initialization
ret, img = cap.read()
h, w, c = img.shape

img_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
img_GpuMat.create((w, h), cv2.CV_8UC3) # cv2.CV_8UC3 -> 8bit image 3 channel
hsv_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
hsv_GpuMat.create((w, h), cv2.CV_8UC3) # cv2.CV_8UC3 -> 8bit image 3 channel
h_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
h_GpuMat.create((w, h), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
s_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
s_GpuMat.create((w, h), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
v_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
v_GpuMat.create((w, h), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
mask_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
mask_GpuMat.create((w, h), cv2.CV_8UC1) # cv2.CV_8UC1 -> 8bit image 1 channel
res_GpuMat = cv2.cuda_GpuMat() # Create GpuMat object 
res_GpuMat.create((w, h), cv2.CV_8UC3) # cv2.CV_8UC3 -> 8bit image 3 channel

# create CUDA eroding morphological transform object with kernel 3x3
kernel = np.ones((2,2),np.uint8)
MorphObj = cv2.cuda.createMorphologyFilter(cv2.MORPH_ERODE, cv2.CV_8UC1, kernel, iterations = 1)

times = []
while cap.isOpened() :
    object_contour = {}
    object_count = {}
    object_id = 0 

    e1 = cv2.getTickCount()
    ret, img = cap.read()
    if not ret : 
        break 

    # upload to GPU memory
    img_GpuMat.upload(img)

    # CUDA convert to hsv
    cv2.cuda.cvtColor(img_GpuMat, cv2.COLOR_BGR2HSV, hsv_GpuMat)

    # split HSV GPU Mat
    cv2.cuda.split(hsv_GpuMat, [h_GpuMat, s_GpuMat, v_GpuMat])

    # CUDA Threshold the V(20,150) ~ HSV GPU Mat to get only gray colors
    cv2.cuda.inRange(v_GpuMat, 20, 150, mask_GpuMat)

    # CUDA Eroding Morphological Transform
    MorphObj.apply(mask_GpuMat, mask_GpuMat)

    # CUDA bitwise operation
    cv2.cuda.bitwise_not(img_GpuMat, res_GpuMat, mask=mask_GpuMat) # apply bitwise NOT to original image -> result image
    cv2.cuda.bitwise_not(res_GpuMat, res_GpuMat, mask=mask_GpuMat) # apply bitwise NOT to result image 

    # Download Matrix to Host Memory                                                               
    mask = mask_GpuMat.download()
    res = res_GpuMat.download()

    # find contour from range thresholding
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    for cnt, hrcy in zip(contours, hierarchy[0]):
        # find contour Area & boungin Rect
        area = cv2.contourArea(cnt)
        x, y, w, h = cv2.boundingRect(cnt)

        # calculate aspectRatio & extent
        aspectRatio = float(w)/h 
        rect_area = w*h
        extent = float(area)/rect_area
        
        # filter a small contour
        if area <= MIN_AREA:
            continue 
        
        # Find All Extreme Outer Contour [BLUE]
        if hrcy[3] == -1 :   
            if aspectRatio < MAX_ASPECT_RATIO and aspectRatio > MIN_ASPECT_RATIO and extent > MIN_EXTENT:      
                cv2.drawContours(res, cnt, -1, (255,0,0), 2)
                
                object_contour["object_%d" % object_id] = cnt # insert parent contour
                object_count["object_%d" % object_id] = 0 # set initinal count 0
                object_id += 1

        # Find All child contour [GREEN]
        if hrcy[3] != -1 :  
            if aspectRatio < MAX_ASPECT_RATIO_CHILD and aspectRatio > MIN_ASPECT_RATIO_CHILD and extent > MIN_EXTENT_CHILD:    
                cv2.drawContours(res, cnt, -1, (0,255,0), 2)

                for obj_name in object_contour:
                    # find the child contour on wich parrent contour
                    if cv2.pointPolygonTest(object_contour[obj_name], (x, y), measureDist=True) > 0 :
                        object_count[obj_name] += 1


    for obj_name in object_count:
        x, y, w, h = cv2.boundingRect(object_contour[obj_name])
        # check if number of child contour inside parrent less than threshold count 
        if object_count[obj_name] < THRESHOLD_COUNT :
            img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                        font_size=0.4, alpha=0.6, bg_color=(0,0,255), ouline_color=(0,0,255), text_color=(0,0,0))
        else :
            img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                        font_size=0.4, alpha=0.6, bg_color=(0,255,0), ouline_color=(0,255,0), text_color=(0,0,0)) 

    cv2.imshow(window_name_1, img)
    cv2.imshow(window_name_2, res)
    if cv2.waitKey(1) == ord("q"):
        break
    e2 = cv2.getTickCount()
    times.append((e2 - e1)/ cv2.getTickFrequency())

time_avg = np.array(times).mean()
print("Average execution time : %.4fs" % time_avg)
print("Average FPS : %.2f" % (1/time_avg))

cv2.destroyAllWindows()


## Result Benchmark
- CUDA Contour Based Visual Inspection Engine for Video Stream (FFMPEG Backend) + Jetson Maximized Performance
    - Average execution time : 0.3222s
    - Average FPS : 3.10
- CUDA Contour Based Visual Inspection Engine for Video Stream (GStreamer Backend) + Jetson Maximized Performance
    - Average execution time : 0.2777s
    - Average FPS : 3.60
- CUDA Contour Based Visual Inspection Engine for Video Stream (FFMPEG Backend) + Jetson Maximized Performance + OpenGL Image Rendering
    - Average execution time : 0.0654s
    - Average FPS : 15.30

___
# 4. Object Segmentation using OpenCV Tracking API

In [3]:
# load GStreamer File Loader 
from gst_file import gst_file_loader

# load video file using GStreamer 
cap = cv2.VideoCapture(gst_file_loader("video_a.mp4", useRotate90=True), cv2.CAP_GSTREAMER)  

# Choose tracker
#tracker = cv2.TrackerCSRT_create()
tracker = cv2.TrackerKCF_create()

___, img = cap.read()

# create initial bounding box
bbox = cv2.selectROI("Tracking",img,False)

tracker.init(img, bbox)

while cap.isOpened():
    e1 = cv2.getTickCount()
    ret, img = cap.read()
    
    if not ret : 
        break

    success, bbox = tracker.update(img)

    if success:
        # draw bounding box
        x ,y ,w ,h = np.int0(bbox)
        cv2.rectangle(img, (x, y), (x+w, y+h), (255,0,255), 3, 1)
        cv2.putText(img, "Tracking", (75, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 1)
    else:
        cv2.putText(img,"Lost", (75, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 1)
    
    e2 = cv2.getTickCount()
    fps = cv2.getTickFrequency()/(e2-e1)
    
    cv2.putText(img,"%d FPS " % fps, (75,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 1)
    cv2.imshow("Tracking",img)

    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()

___
## 4.2 Combine Contour Based + Tracking API

In [ ]:
# EXAMPLE 4.2.1 | Apply OpenCV Tracking API + Contour Based Visual Inspection Engine for Video Stream

# Uncomment this if using OpenGL for image rendering
window_name_1 = "Detected Object"
window_name_2 = "Detected Contour"
# cv2.namedWindow(window_name_1, flags=cv2.WINDOW_OPENGL)    # Window with OpenGL
# cv2.namedWindow(window_name_2, flags=cv2.WINDOW_OPENGL)    # Window with OpenGL

# define range of gray color in HSV 
lower_gray = np.array([0, 0, 20])       # (part_a.jpg : [0, 0, 20] | part_b.jpg : [0, 0, 10])
upper_gray = np.array([180, 100, 150])  # (part_a.jpg : [180, 100, 150]| part_b.jpg : [180, 100, 170])

THRESHOLD_COUNT = 22 # min number of child contour  (part_a.jpg : 22 | part_b.jpg : 2)
MIN_AREA = 100 # minimum number of pixel to be counted to reject small contour

# Contour Property parameter for parent contour
MAX_ASPECT_RATIO = 0.3 # (part_a.jpg : 0.3 | part_b.jpg : 0.5)
MIN_ASPECT_RATIO = 0.1 # (part_a.jpg : 0.1 | part_b.jpg : 0.3)
MIN_EXTENT = 0.4

# Contour Property parameter for child contour
MAX_ASPECT_RATIO_CHILD = 1.5
MIN_ASPECT_RATIO_CHILD = 0.5
MIN_EXTENT_CHILD = 0.4


# load video file using GStreamer
# cap = cv2.VideoCapture(gst_file_loader("video_a.mp4", useRotate90=True), cv2.CAP_GSTREAMER)  # backend GSTREAMER 

cap = cv2.VideoCapture("video_a.mp4", cv2.CAP_FFMPEG)  # backend FFMPEG
cap.set(cv2.CAP_PROP_ORIENTATION_AUTO, 1) # applicable for backend FFMPEG only

# Initialize OpenCV Tracking API
tracker_green = cv2.TrackerKCF_create()
tracker_red = cv2.TrackerKCF_create()
is_tracker_green_available = False
is_tracker_red_available = False
print("Tracking started...")


times = []
while cap.isOpened() :
    object_contour = {}
    object_count = {}
    object_id = 0 

    e1 = cv2.getTickCount()
    ret, img = cap.read()
    if not ret : 
        break 

    if not (is_tracker_green_available or is_tracker_red_available):
        #convert to hsv
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        
        # Threshold the HSV image to get only gray colors
        mask = cv2.inRange(hsv.copy(), lower_gray, upper_gray)

        # apply eroding into mask
        kernel = np.ones((2,2),np.uint8)
        erosion = cv2.erode(mask, kernel, iterations = 2)

        # apply bitwise operation (for background removal), if needed.
        res = cv2.bitwise_and(img, img, mask=erosion)

        # find contour from range thresholding
        contours, hierarchy = cv2.findContours(erosion, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

        for cnt, hrcy in zip(contours, hierarchy[0]):
            # find contour Area & boungin Rect
            area = cv2.contourArea(cnt)
            x, y, w, h = cv2.boundingRect(cnt)

            # calculate aspectRatio & extent
            aspectRatio = float(w)/h 
            rect_area = w*h
            extent = float(area)/rect_area
            
            # filter a small contour
            if area <= MIN_AREA:
                continue 
            
            # Find All Extreme Outer Contour [BLUE]
            if hrcy[3] == -1 :   
                if aspectRatio < MAX_ASPECT_RATIO and aspectRatio > MIN_ASPECT_RATIO and extent > MIN_EXTENT:      
                    cv2.drawContours(res, cnt, -1, (255,0,0), 2)
                    
                    object_contour["object_%d" % object_id] = cnt # insert parent contour
                    object_count["object_%d" % object_id] = 0 # set initinal count 0
                    object_id += 1

            # Find All child contour [GREEN]
            if hrcy[3] != -1 :  
                if aspectRatio < MAX_ASPECT_RATIO_CHILD and aspectRatio > MIN_ASPECT_RATIO_CHILD and extent > MIN_EXTENT_CHILD:    
                    cv2.drawContours(res, cnt, -1, (0,255,0), 2)

                    for obj_name in object_contour:
                        # find the child contour on wich parrent contour
                        if cv2.pointPolygonTest(object_contour[obj_name], (x, y), measureDist=True) > 0 :
                            object_count[obj_name] += 1


        for obj_name in object_count:
            x, y, w, h = cv2.boundingRect(object_contour[obj_name])
            # check if number of child contour inside parrent less than threshold count 
            if object_count[obj_name] < THRESHOLD_COUNT :
                try :
                    tracker_red.init(img, [x, y, w, h])
                    is_tracker_red_available = True
                except : 
                    pass
                img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                            font_size=0.4, alpha=0.6, bg_color=(0,0,255), ouline_color=(0,0,255), text_color=(0,0,0))
            else :
                try :
                    tracker_green.init(img, [x, y, w, h])
                    is_tracker_green_available = True
                except : 
                    pass
                img = draw_ped(img, "%s (%d)" % (obj_name, object_count[obj_name])  , x, y, x+w, y+h, 
                            font_size=0.4, alpha=0.6, bg_color=(0,255,0), ouline_color=(0,255,0), text_color=(0,0,0)) 

    elif is_tracker_red_available : 
        try :
            is_tracker_red_available, bbox_red = tracker_red.update(img)
            if is_tracker_red_available :
                x ,y ,w ,h = np.int0(bbox_red)
                img = draw_ped(img, "Tracking - 1"  , x, y, x+w, y+h, 
                            font_size=0.4, alpha=0.6, bg_color=(0,0,255), ouline_color=(0,0,255), text_color=(0,0,0))
        except : 
            pass
        
    elif is_tracker_green_available : 
        try :
            is_tracker_green_available, bbox_green = tracker_green.update(img)
            if is_tracker_green_available :
                x ,y ,w ,h = np.int0(bbox_green)
                img = draw_ped(img, "Tracking - 2"  , x, y, x+w, y+h, 
                            font_size=0.4, alpha=0.6, bg_color=(0,0,255), ouline_color=(0,0,255), text_color=(0,0,0))
        except :
            pass

    cv2.imshow(window_name_1, img)
    cv2.imshow(window_name_2, res)
    if cv2.waitKey(1) == ord("q"):
        break

    e2 = cv2.getTickCount()
    times.append((e2 - e1)/ cv2.getTickFrequency())

time_avg = np.array(times).mean()
print("Average execution time : %.4fs" % time_avg)
print("Average FPS : %.2f" % (1/time_avg))

cv2.destroyAllWindows()